# Instalação Libs

In [0]:
%pip install langchain==0.3.25 langchain-google-vertexai langgraph databricks-sql-connector pandas python-dotenv langchain-databricks==0.1.0 

In [0]:
    %pip install langchain-openai

In [0]:
# Forçando atualização de tudo para alinhar as versões
%pip install --upgrade --force-reinstall langchain langchain-core langchain-openai langchain-databricks langchain-google-vertexai langgraph databricks-sql-connector pandas python-dotenv


In [0]:
%pip install langchain-databricks==0.1.0 

# Teste Agent

In [0]:
import os
import sys

# 1. Adicionar o diretório atual ao PATH para importar os módulos 'src'
# Se estiver usando Repos, o caminho geralmente já está correto, mas garante:
current_dir = os.getcwd()
sys.path.append(current_dir)

# 2. Configurar Credenciais do Google (Se usar Vertex AI)
# Ajuste o caminho para onde você subiu seu JSON
#    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/dbfs/FileStore/credentials/seu-arquivo-service-account.json"
#    os.environ["GOOGLE_CLOUD_PROJECT"] = "seu-id-do-projeto-gcp"

DEPLOY_TYPE = os.getenv("DEPLOY_TYPE")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 

# 3. Configurar Credenciais do Databricks (Para o conector SQL)
# Pegue esses valores em Compute -> Advanced Options -> JDBC/ODBC
SERVER_HOSTNAME = os.getenv("DATABRICKS_SERVER_HOSTNAME")
HTTP_PATH = os.getenv("DATABRICKS_HTTP_PATH") # Seu HTTP Path longo
ACCESS_TOKEN = os.getenv("DATABRICKS_TOKEN")  # Seu token gerado 
print('SERVER_HOSTNAME', SERVER_HOSTNAME)
# --- HACK PARA O CÓDIGO ---
# Como o código em 'nodes.py' pode estar hardcoded, podemos injetar ou
# redefinir a instância do serviço aqui, ou editar o arquivo nodes.py diretamente no Repo.

from src.services.databricks import DatabricksService
from examples.walmart_agent.agent import WalmartAgent
import examples.walmart_agent.nodes as agent_nodes

# Sobrescrevendo o serviço hardcoded no módulo nodes (se necessário)
print("Configurando serviço Databricks real...")
agent_nodes.databricks_service = DatabricksService(
      server_hostname=SERVER_HOSTNAME,
      http_path=HTTP_PATH,
      access_token=ACCESS_TOKEN,
      # catalog="hive_metastore", # No Community geralmente usa-se hive_metastore
      # schema="default"          # Ou seu schema criado
)

# 4. Inicializar e Rodar o Agente
print("Inicializando Agente...")
agent = WalmartAgent(project_id="seu-projeto-gcp") # O project_id aqui pode ser irrelevante se já setou a env var

query = "Show me the top 10 sales records"
print(f"Executando query: {query}")

response = agent.chat(query)
print("\nResposta do Agente:")
print(response)

# Deploy Agent

In [0]:
import mlflow
import pandas as pd
import os
from mlflow.pyfunc import PythonModel, PythonModelContext

# 1. Definir o Wrapper do MLflow
class WalmartAgentWrapper(PythonModel):
    def load_context(self, context):
        """
        Este método roda UMA VEZ quando o container do modelo sobe.
        Ideal para carregar arquivos, inicializar classes pesadas, etc.
        """
        # Importação dentro da função para garantir que as dependências estejam instaladas no container
        from examples.walmart_agent.agent import WalmartAgent
        
        # Inicializa o agente
        # Nota: As variáveis de ambiente (API KEYS) serão injetadas pelo Serving Endpoint
        self.agent = WalmartAgent() 

    def predict(self, context, model_input):
        """
        Este método roda a cada request da API.
        model_input geralmente vem como um pandas DataFrame.
        """
        # Extrai a mensagem do input (assumindo que o JSON de entrada tem um campo 'query')
        if isinstance(model_input, pd.DataFrame):
            user_message = model_input.iloc[0]["query"]
        else:
            user_message = model_input.get("query")
            
        # Chama seu agente
        response = self.agent.chat(user_message)
        
        # Retorna lista ou string (formato esperado pelo MLflow)
        return [response]

# 2. Configurar o Experimento
mlflow.set_registry_uri("databricks-uc") # Usar Unity Catalog
catalog_name = "walmart_project" # Seu catálogo
schema_name = "models"           # Seu schema
model_name = f"{catalog_name}.{schema_name}.walmart_sales_agent"

# mlflow.set_experiment(f"/Users/seu_usuario/walmart_agent_experiment")

# 3. Definir Dependências (Copia seu requirements.txt)
# É CRUCIAL listar tudo que seu código usa
pip_requirements = [
    "langchain==0.3.25",
    "langchain-core==0.3.65",
    "langchain-openai",
    "langchain-community",
    "langgraph",
    "databricks-sql-connector",
    "pandas",
    "pydantic"
]

# 4. Logar e Registrar o Modelo
with mlflow.start_run() as run:
    # Assinatura de entrada e saída (Documentação da API)
    input_example = pd.DataFrame({"query": ["Show me top 10 sales"]})
    signature = mlflow.models.infer_signature(
        model_input=input_example,
        model_output=["Resposta do agente..."]
    )

    # Log do modelo com o código fonte
    # 'code_path' deve apontar para a pasta raiz do seu projeto para que os imports (src, examples) funcionem
    mlflow.pyfunc.log_model(
        artifact_path="agent_model",
        python_model=WalmartAgentWrapper(),
        signature=signature,
        input_example=input_example,
        pip_requirements=pip_requirements,
        code_path=["./langgraph_accelerator/"] # Caminho relativo para incluir sua pasta de código
    )

print(f"Modelo logado! Run ID: {run.info.run_id}")

# 5. Registrar no Catálogo (Opcional se fizer via UI, mas bom via código)
model_version = mlflow.register_model(
    model_uri=f"runs:/{run.info.run_id}/agent_model",
    name=model_name
)